In [ ]:
from google.colab import files
uploaded = files.upload()

Saving insurance_policies.csv to insurance_policies.csv


In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
summarization = pd.read_csv('/content/insurance_policies.csv')
summarization.head()

,Policy_ID,Policy_Text_EN
0,5ea4d7cd-a834-48b8-9b80-5d5b4e79263b,This comprehensive insurance policy offers cov...
1,369b1a12-b8fa-4235-8435-b0ff839797db,The health insurance plan offers an all-inclus...
2,9cb6703b-9ef3-4692-ae38-8da75a197760,This motor insurance plan provides comprehensi...
3,9964fb8e-2523-4147-ac66-baf944c88595,Travel insurance provides worldwide coverage f...
4,1e4a9661-9d57-472c-ab67-1c4d29f106e1,This group insurance policy is designed for co...


In [ ]:
pip install nltk

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

def clean_text(text):
    # Lowercase text
    text = text.lower()

    # Remove non-alphanumeric characters
    text = re.sub(r'[^a-zA-Z0-9\s-]', '', text)

    # Tokenize and remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatize words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Return cleaned text
    return ' '.join(tokens)

In [ ]:
summarization['Policy_Text_Cleaned'] = summarization['Policy_Text_EN'].apply(clean_text)
summarization

,Policy_ID,Policy_Text_EN,Policy_Text_Cleaned
0,5ea4d7cd-a834-48b8-9b80-5d5b4e79263b,This comprehensive insurance policy offers cov...,comprehensive insurance policy offer coverage ...
1,369b1a12-b8fa-4235-8435-b0ff839797db,The health insurance plan offers an all-inclus...,health insurance plan offer all-inclusive medi...
2,9cb6703b-9ef3-4692-ae38-8da75a197760,This motor insurance plan provides comprehensi...,motor insurance plan provides comprehensive pr...
3,9964fb8e-2523-4147-ac66-baf944c88595,Travel insurance provides worldwide coverage f...,travel insurance provides worldwide coverage m...
4,1e4a9661-9d57-472c-ab67-1c4d29f106e1,This group insurance policy is designed for co...,group insurance policy designed corporates pro...
...,...,...,...
695,532866aa-71b5-454a-99f8-cb63e9b8043e,The health insurance plan offers an all-inclus...,health insurance plan offer all-inclusive medi...
696,e6aa7ef3-77e4-4856-9ead-8560ba9250a7,This group insurance policy is designed for co...,group insurance policy designed corporates pro...
697,da5b72ef-181c-49ed-9ae4-73fb1b760f9d,This comprehensive insurance policy delivers c...,comprehensive insurance policy delivers covera...
698,27cf0252-70b3-4f76-924f-ef5f5ac78053,This motor insurance plan provides comprehensi...,motor insurance plan provides comprehensive pr...


In [ ]:

from transformers import MT5ForConditionalGeneration, MT5Tokenizer
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer  = MT5Tokenizer.from_pretrained(model_name)
model      = MT5ForConditionalGeneration.from_pretrained(model_name).to(device)

def mt5_summarize(text: str) -> str:
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=512
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.generate(
        **inputs,
        max_length=250,
        min_length=60,
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

summarization['Summary'] = summarization['Policy_Text_Cleaned'].apply(mt5_summarize)



tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'MT5Tokenizer'.
You are using the default legacy behaviour of the <class 'transformers.models.mt5.tokenization_mt5.MT5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

In [ ]:
# Assuming summarization DataFrame is already created and populated with 'Summary' column
sample_indices = [0, 1, 2, 3, 4]  # Indices of the 5 sample sentences to compare
for i in sample_indices:
    print(f"Original Text (Sample {i + 1}):\n{summarization['Policy_Text_EN'][i]}\n")
    print(f"Summary (Sample {i + 1}):\n{summarization['Summary'][i]}\n")
    print("-" * 50)  # Separator between samples


Original Text (Sample 1):
This comprehensive insurance policy offers coverage for a variety of risks associated with residential and commercial properties. It includes protection against fire, theft, burglary, floods, earthquakes, and other natural calamities. The policy also covers accidental damage and liability arising from third-party claims. Policyholders are required to maintain accurate documentation of their property’s value and condition, submit claims within 48 hours of the incident, and ensure regular maintenance to prevent damage. The insurer reserves the right to inspect the property before issuing the policy. Failure to comply with any clause may result in partial or full denial of claims. Add-ons such as rental protection, alternative accommodation reimbursement, and electronic equipment insurance are also available under this plan. Annual premium varies based on property size, location, and coverage amount.

Summary (Sample 1):
Consumers are being offered a range of ins

In [ ]:
summarization

,Policy_ID,Policy_Text_EN,Policy_Text_Cleaned,Summary
0,5ea4d7cd-a834-48b8-9b80-5d5b4e79263b,This comprehensive insurance policy offers cov...,comprehensive insurance policy offer coverage ...,Consumers are being offered a range of insuran...
1,369b1a12-b8fa-4235-8435-b0ff839797db,The health insurance plan offers an all-inclus...,health insurance plan offer all-inclusive medi...,A list of thousands of health insurance claims...
2,9cb6703b-9ef3-4692-ae38-8da75a197760,This motor insurance plan provides comprehensi...,motor insurance plan provides comprehensive pr...,Motor insurance claims are being submitted to ...
3,9964fb8e-2523-4147-ac66-baf944c88595,Travel insurance provides worldwide coverage f...,travel insurance provides worldwide coverage m...,Passengers who travel to and from across the w...
4,1e4a9661-9d57-472c-ab67-1c4d29f106e1,This group insurance policy is designed for co...,group insurance policy designed corporates pro...,Companies which provide life health accident c...
...,...,...,...,...
695,532866aa-71b5-454a-99f8-cb63e9b8043e,The health insurance plan offers an all-inclus...,health insurance plan offer all-inclusive medi...,A list of thousands of health insurance claims...
696,e6aa7ef3-77e4-4856-9ead-8560ba9250a7,This group insurance policy is designed for co...,group insurance policy designed corporates pro...,Companies which provide life health accident c...
697,da5b72ef-181c-49ed-9ae4-73fb1b760f9d,This comprehensive insurance policy delivers c...,comprehensive insurance policy delivers covera...,Comprehensive insurance is being offered to an...
698,27cf0252-70b3-4f76-924f-ef5f5ac78053,This motor insurance plan provides comprehensi...,motor insurance plan provides comprehensive pr...,Motor insurance claims are being submitted to ...


In [ ]:
summarization.to_csv("summarized_policies.csv", index=False, encoding="utf-8")
print("Summarization completed and saved as summarized_policies.csv")

# files.download("summarized_policies.csv")

Summarization completed and saved as summarized_policies.csv


In [ ]:
!pip install evaluate
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.


In [ ]:
pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=01f485b89e8e6e833b265434aa8eb9762987742a7199314f356dca721e9b2e2b
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
import warnings
warnings.filterwarnings('ignore')


In [ ]:

from transformers import MT5ForConditionalGeneration, MT5Tokenizer
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer  = MT5Tokenizer.from_pretrained(model_name)
model      = MT5ForConditionalGeneration.from_pretrained(model_name).to(device)

tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'MT5Tokenizer'.
You are using the default legacy behaviour of the <class 'transformers.models.mt5.tokenization_mt5.MT5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

In [ ]:
from google.colab import files  # For manual file upload
import pandas as pd

# 🔹 Step 1: Upload CSV File Manually
uploaded = files.upload()  # Opens a file selection dialog

# Get the uploaded filename
filename = list(uploaded.keys())[0]
# Load the dataset
summarization = pd.read_csv(filename)

Saving summarized_policies.csv to summarized_policies.csv


In [ ]:
# Convert to Hugging Face Dataset

from datasets import Dataset
import evaluate

dataset = Dataset.from_pandas(summarization[['Policy_Text_EN', 'Summary']])

# Split into train/test
dataset = dataset.train_test_split(test_size=0.1)
print(f"✅ Dataset split: {len(dataset['train'])} train | {len(dataset['test'])} test")



✅ Dataset split: 630 train | 70 test


In [ ]:
# 🔹 Step 3: Preprocessing Function
def preprocess_function(examples):
    inputs = tokenizer(
        examples["Policy_Text_EN"], padding="max_length", truncation=True, max_length=512
    )
    targets = tokenizer(
        examples["Summary"], padding="max_length", truncation=True, max_length=128
    )
    inputs["labels"] = targets["input_ids"]
    return inputs


In [ ]:
# Tokenize dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["Policy_Text_EN", "Summary"])
print(f"✅ Tokenization completed!")

Map:   0%|          | 0/630 [00:00<?, ? examples/s]

Map:   0%|          | 0/70 [00:00<?, ? examples/s]

✅ Tokenization completed!


In [ ]:
# 🔹 Step 4: Define Training Arguments
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    load_best_model_at_end=True,  # Critical for saving best model
    metric_for_best_model="rougeL",
    greater_is_better=True,  # Higher ROUGE is better
    predict_with_generate=True,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    fp16=torch.cuda.is_available(),
    logging_dir="./logs",
    logging_steps=500,
    report_to="none",  # Disable reporting (e.g., WandB, TensorBoard)
)

In [ ]:
# 🔹 Step 5: Define Trainer
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    return {k: round(v * 100, 2) for k, v in result.items()}


In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# 🔹 Step 6: Train Model
print("🚀 Starting training...")
trainer.train()
print("✅ Training completed!")


🚀 Starting training...


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,No log,0.050395,79.800000,69.560000,74.870000,74.610000
2,No log,0.046754,79.770000,68.980000,74.750000,74.400000
3,No log,0.046763,79.770000,68.980000,74.750000,74.400000


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


✅ Training completed!


In [ ]:
# 🔹 Step 7: Save Model
model.save_pretrained("./summarized_fine_tuned_model")
tokenizer.save_pretrained("./summarized_fine_tuned_model")
print("✅ Fine-tuning completed and model saved to './summarized_fine_tuned_model'")

✅ Fine-tuning completed and model saved to './summarized_fine_tuned_model'


In [ ]:
# 🔹 Step 9: Zip and Download Model
import shutil

model_dir = "./summarized_fine_tuned_model"
shutil.make_archive("summarized_fine_tuned_model", 'zip', model_dir)  # Create a ZIP file
files.download("summarized_fine_tuned_model.zip")  # Download the ZIP file
print("✅ Model zipped and ready for download!")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Model zipped and ready for download!
